# PEC 2: gráfico Ridgeline

Se cargan las librerías necesarias

In [10]:
import matplotlib.pyplot as plt
import kagglehub as khub
import pandas as pd

Se cargan los datos desde kaggle

In [13]:
path = khub.dataset_download("sudalairajkumar/daily-temperature-of-major-cities")
df = pd.read_csv(path+"/city_temperature.csv")
print(df.info())

C:\Users\jgonzalez25D\AppData\Local\Temp\ipykernel_7172\1985251444.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path+"/city_temperature.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2906327 entries, 0 to 2906326
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Region          object 
 1   Country         object 
 2   State           object 
 3   City            object 
 4   Month           int64  
 5   Day             int64  
 6   Year            int64  
 7   AvgTemperature  float64
dtypes: float64(1), int64(3), object(4)
memory usage: 177.4+ MB
None


In [27]:
df[(df['City'] == 'Barcelona')]

,Region,Country,State,City,Month,Day,Year,AvgTemperature
903441,Europe,Spain,NaN,Barcelona,1,1,1995,50.8
903442,Europe,Spain,NaN,Barcelona,1,2,1995,44.5
903443,Europe,Spain,NaN,Barcelona,1,3,1995,41.0
903444,Europe,Spain,NaN,Barcelona,1,4,1995,39.2
903445,Europe,Spain,NaN,Barcelona,1,5,1995,39.8
...,...,...,...,...,...,...,...,...
912702,Europe,Spain,NaN,Barcelona,5,9,2020,65.7
912703,Europe,Spain,NaN,Barcelona,5,10,2020,64.1
912704,Europe,Spain,NaN,Barcelona,5,11,2020,65.6
912705,Europe,Spain,NaN,Barcelona,5,12,2020,63.4
